# Import Libraries

In [1]:
from sklearn.datasets import fetch_20newsgroups
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM,Embedding,Dense,Conv1D,Dropout,MaxPooling1D,Flatten
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

Using TensorFlow backend.


# Preprocessing

### Initialize Dataset

In [2]:
train_text = fetch_20newsgroups(subset='train').data
train_label = fetch_20newsgroups(subset='train').target
test_text = fetch_20newsgroups(subset='test').data
test_label = fetch_20newsgroups(subset='test').target

### Initialize Constatns

In [3]:
max_words_to_keep = 5000
maxlen_text = 200
token_vec_size = 128
output_dim = np.unique(train_label).__len__()

### Tokenize and pad each newsgroups posts 

In [4]:
tokenizer = Tokenizer(num_words=max_words_to_keep,filters='!"#$%&()*+,\'-./:;<=>?@[\\]^_`{|}~\t\n\"',
                        lower=True,
                        split=" ",
                        char_level=False)
tokenizer.fit_on_texts(train_text)

sequences = tokenizer.texts_to_sequences(train_text)
train_X = pad_sequences(sequences=sequences,maxlen=maxlen_text)

sequences = tokenizer.texts_to_sequences(test_text)
test_X = pad_sequences(sequences=sequences,maxlen=maxlen_text)

### Handle the Label using LabelEncoder

In [5]:
encoder = LabelEncoder()

encoder.fit(train_label)

encoded_train_Y = encoder.transform(train_label)
train_Y = np_utils.to_categorical(encoded_train_Y,num_classes=output_dim)

encoded_train_Y = encoder.transform(test_label)
test_Y = np_utils.to_categorical(encoded_train_Y,num_classes=output_dim)

### Lets print the shapes of data and labels

In [6]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

(11314, 200)
(11314, 20)
(7532, 200)
(7532, 20)


# Spatial Modeling

### Define a CNN model

In [7]:
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=max_words_to_keep, output_dim=token_vec_size, input_length=maxlen_text))
cnn_model.add(Dropout(0.2))
cnn_model.add(Conv1D(64, 5, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=4))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=output_dim,activation='softmax'))

### Compile the CNN model

In [8]:
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the CNN model

In [9]:
cnn_model.fit(train_X, train_Y, batch_size=1000,validation_split=0.3, epochs=20)

Train on 7919 samples, validate on 3395 samples
Epoch 1/20
7919/7919 [==============================] - 5s - loss: 2.9804 - acc: 0.0638 - val_loss: 2.9574 - val_acc: 0.0834
Epoch 2/20
7919/7919 [==============================] - 0s - loss: 2.9139 - acc: 0.1217 - val_loss: 2.9242 - val_acc: 0.1037
Epoch 3/20
7919/7919 [==============================] - 0s - loss: 2.8723 - acc: 0.1505 - val_loss: 2.8941 - val_acc: 0.1090
Epoch 4/20
7919/7919 [==============================] - 0s - loss: 2.8085 - acc: 0.1818 - val_loss: 2.8370 - val_acc: 0.1517
Epoch 5/20
7919/7919 [==============================] - 0s - loss: 2.7068 - acc: 0.2551 - val_loss: 2.7245 - val_acc: 0.1770
Epoch 6/20
7919/7919 [==============================] - 0s - loss: 2.5352 - acc: 0.3359 - val_loss: 2.5394 - val_acc: 0.2359
Epoch 7/20
7919/7919 [==============================] - 0s - loss: 2.3014 - acc: 0.4019 - val_loss: 2.3255 - val_acc: 0.2931
Epoch 8/20
7919/7919 [==============================] - 0s - loss: 2.0337 - a

### Evaluate the prediction on the Test Data

In [10]:
res_class_cnn = cnn_model.predict_classes(test_X)
print(res_class_cnn)

6944/7532 [==========================>...] - ETA: 0s[12  1 19 ...,  7  4 15]


In [11]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(test_X,test_Y,verbose=2)
print(cnn_loss)
print(cnn_accuracy)

1.24981001622
0.605549654806


# Temporal Modeling

### Define an LSTM model

In [12]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=max_words_to_keep, output_dim=token_vec_size, input_length=maxlen_text))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(units=token_vec_size, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(units=output_dim, activation='softmax'))

### Compile the LSTM model

In [13]:
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the LSTM model

In [14]:
lstm_model.fit(train_X, train_Y, batch_size=1000,validation_split=0.3, epochs=20)

Train on 7919 samples, validate on 3395 samples
Epoch 1/20
7919/7919 [==============================] - 5s - loss: 2.9922 - acc: 0.0787 - val_loss: 2.9870 - val_acc: 0.1046
Epoch 2/20
7919/7919 [==============================] - 4s - loss: 2.9741 - acc: 0.1443 - val_loss: 2.9618 - val_acc: 0.0890
Epoch 3/20
7919/7919 [==============================] - 5s - loss: 2.9225 - acc: 0.1874 - val_loss: 2.8828 - val_acc: 0.1803
Epoch 4/20
7919/7919 [==============================] - 4s - loss: 2.7972 - acc: 0.1662 - val_loss: 2.7051 - val_acc: 0.1370
Epoch 5/20
7919/7919 [==============================] - 4s - loss: 2.6400 - acc: 0.1673 - val_loss: 2.6035 - val_acc: 0.1405
Epoch 6/20
7919/7919 [==============================] - 4s - loss: 2.5404 - acc: 0.1841 - val_loss: 2.5186 - val_acc: 0.2041
Epoch 7/20
7919/7919 [==============================] - 4s - loss: 2.4534 - acc: 0.2339 - val_loss: 2.4411 - val_acc: 0.2218
Epoch 8/20
7919/7919 [==============================] - 4s - loss: 2.3486 - a

### Evaluate the prediction on the test data

In [15]:
res_class_lstm = lstm_model.predict_classes(test_X)
print(res_class_lstm)

7532/7532 [==============================] - 16s    
[ 7  6  0 ..., 11  6 15]


In [16]:
loss_lstm, accuracy_lstm = lstm_model.evaluate(test_X,test_Y,verbose=2)
print(loss_lstm)
print(accuracy_lstm)

1.79106456804
0.412904938927


# Spatio-Temporal Model

### Define a hybrid of CNN and LSTM

In [17]:
hypbrid_model = Sequential()
hypbrid_model.add(Embedding(input_dim=max_words_to_keep, output_dim=token_vec_size, input_length=maxlen_text))
hypbrid_model.add(Dropout(0.2))
hypbrid_model.add(Conv1D(64, 5, activation='relu'))
hypbrid_model.add(MaxPooling1D(pool_size=4))
hypbrid_model.add(LSTM(128))
hypbrid_model.add(Dense(units=output_dim, activation='softmax'))

### Compile the Hybrid model

In [18]:
hypbrid_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Train the Hybrid model

In [19]:
hypbrid_model.fit(train_X, train_Y, batch_size=1000,validation_split=0.3, epochs=20)

Train on 7919 samples, validate on 3395 samples
Epoch 1/20
7919/7919 [==============================] - 2s - loss: 2.9918 - acc: 0.0657 - val_loss: 2.9878 - val_acc: 0.0934
Epoch 2/20
7919/7919 [==============================] - 1s - loss: 2.9764 - acc: 0.1118 - val_loss: 2.9736 - val_acc: 0.1025
Epoch 3/20
7919/7919 [==============================] - 1s - loss: 2.9332 - acc: 0.1320 - val_loss: 2.9112 - val_acc: 0.1187
Epoch 4/20
7919/7919 [==============================] - 1s - loss: 2.8253 - acc: 0.1443 - val_loss: 2.7323 - val_acc: 0.1711
Epoch 5/20
7919/7919 [==============================] - 1s - loss: 2.5236 - acc: 0.1796 - val_loss: 2.4187 - val_acc: 0.1876
Epoch 6/20
7919/7919 [==============================] - 1s - loss: 2.2435 - acc: 0.2296 - val_loss: 2.1854 - val_acc: 0.2430
Epoch 7/20
7919/7919 [==============================] - 1s - loss: 2.0225 - acc: 0.3055 - val_loss: 1.9887 - val_acc: 0.3308
Epoch 8/20
7919/7919 [==============================] - 1s - loss: 1.7684 - a

### Evaluate the prediction on the Test Data

In [20]:
res_class_hybrid = hypbrid_model.predict_classes(test_X)
print(res_class_hybrid)

7520/7532 [============================>.] - ETA: 0s[ 7  5  0 ..., 18  6 15]


In [21]:
loss_hybrid, accuracy_hybrid = hypbrid_model.evaluate(test_X,test_Y,verbose=2)
print(loss_hybrid)
print(accuracy_hybrid)

1.53089001023
0.573685608136


# Spatio-Temporal Modeling (with multiple tags as output)

### Define,Compile and Train the Model (use sigmoid instead softmax at the last layer)

In [22]:
hybrid_model_mul_tag = Sequential()
hybrid_model_mul_tag.add(Embedding(input_dim=max_words_to_keep, output_dim=token_vec_size, input_length=maxlen_text))
hybrid_model_mul_tag.add(Dropout(0.2))
hybrid_model_mul_tag.add(Conv1D(64, 5, activation='relu'))
hybrid_model_mul_tag.add(MaxPooling1D(pool_size=4))
hybrid_model_mul_tag.add(LSTM(128))
hybrid_model_mul_tag.add(Dense(units=output_dim, activation='sigmoid'))

hybrid_model_mul_tag.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

hybrid_model_mul_tag.fit(train_X, train_Y, batch_size=1000,validation_split=0.3, epochs=20)

Train on 7919 samples, validate on 3395 samples
Epoch 1/20
7919/7919 [==============================] - 2s - loss: 2.9936 - acc: 0.0547 - val_loss: 2.9907 - val_acc: 0.0530
Epoch 2/20
7919/7919 [==============================] - 1s - loss: 2.9839 - acc: 0.0597 - val_loss: 2.9821 - val_acc: 0.0975
Epoch 3/20
7919/7919 [==============================] - 1s - loss: 2.9570 - acc: 0.1008 - val_loss: 2.9385 - val_acc: 0.1249
Epoch 4/20
7919/7919 [==============================] - 1s - loss: 2.8977 - acc: 0.1357 - val_loss: 2.8430 - val_acc: 0.1420
Epoch 5/20
7919/7919 [==============================] - 1s - loss: 2.7327 - acc: 0.1565 - val_loss: 2.6327 - val_acc: 0.1287
Epoch 6/20
7919/7919 [==============================] - 1s - loss: 2.5638 - acc: 0.1308 - val_loss: 2.5639 - val_acc: 0.1084
Epoch 7/20
7919/7919 [==============================] - 1s - loss: 2.4745 - acc: 0.1371 - val_loss: 2.4479 - val_acc: 0.1264
Epoch 8/20
7919/7919 [==============================] - 1s - loss: 2.3800 - a

### Select the tags for which the prob is greater than 0.2 and use the encoder to map it back to the original tags

In [26]:
threshold_tag_prob = 0.2
res_class_prob_hybrid_model_mul_tag = hybrid_model_mul_tag.predict(test_X) > threshold_tag_prob
res_class_hybrid_model_mul_tag = np.array([encoder.inverse_transform(res_class_prob_hybrid_model_mul_tag[i]) for i in np.ndindex(res_class_prob_hybrid_model_mul_tag.shape[:1])])
print(res_class_hybrid_model_mul_tag)

[array([12]) array([1, 3, 4, 5]) array([ 0, 15, 17, 19]) ...,
 array([], dtype=int64) array([2]) array([ 0, 15, 19])]
